In [33]:
%run -i bestestimator_v2.py

In [37]:
import pandas as pd
import numpy as np
import operator

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor, \
    RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelBinarizer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import roc_auc_score


def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return (roc_auc_score(y_test, y_pred, average=average))


class BestEstimator(object):

    def __init__(self, 
                 type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3):
        
        self.type_esti = type_esti
        #self.params = params
        self.cv = cv
        self.grid = grid
        self.hard_grid = hard_grid
        self.cv_grid = cv_grid

        #self.Data = Data.copy()
        #self.Target = Target.copy()
        #self.dim_ = Data.shape
        
        #self.scoring = scoring
        
        self.AUC = make_scorer(multiclass_roc_auc_score)

            #self.scoring = make_scorer(multiclass_roc_auc_score)
        

    def fit(self, data, target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 1000,
            value_nan = 0,
            view_nan = True,
           params = False,
           value = 0,
           scoring = 'AUC'):
        
        loss = scoring 
        Data = data.copy()
        Target = target.copy()

        Data.drop([ID], axis=1, inplace=True)
        if target_ID:
            Target.drop([ID], axis=1, inplace=True)

        if view_nan:
            print("Missing Values :\n")

            total = Data.isnull().sum().sort_values(ascending=False)
            percent = (Data.isnull().sum() / Data.isnull().count()).sort_values(ascending=False) * 100
            missing_data = pd.concat([total, percent], axis=1, keys=['Total', '%'])
            print("{} \n".format(missing_data[(percent > 0)]))

        if type(value) == int:
            Data.fillna(value, inplace=True)
            # self.Test.fillna(value, inplace = True)
            # self.Missing_values()

        elif value == 'bfill':
            Data.fillna('bfill', inplace=True)
            # self.Test.fillna('bfill', inplace = True)
            # self.Missing_values()

        elif value == 'ffill':
            Data.fillna('ffill', inplace=True)
            # self.Test.fillna('ffill', inplace = True)
            # self.Missing_values()

        if Data.isnull().any().any() == False:
            print('NaN data filled by {} \n'.format(value))
        else:
            print('Fail to fill NaN data')

        for i in Data.columns:  ###########

            if Data[i].dtype == object:
                encoder = LabelEncoder()
                encoder.fit(list(Data[i]))
                Data[i] = encoder.transform(list(Data[i]))

            if Data[i].dtype == float:
                Data[i] = Data[i].astype('int')

        for i in Target.columns:
            if Target[i].dtype == object:
                le = LabelEncoder()
                le.fit(list(Target[i]))
                Target[i] = le.transform(list(Target[i]))

        X_tr, X_te, Y_tr, Y_te = train_test_split(Data, Target, random_state=0, test_size=1 / 3)

        print('Searching for the best regressor on {} data using {} loss... \n'.format(n, scoring))

        if self.type_esti == 'classifier':

            # print('\n Searching for the best classifier on {} data... \n'.format(n))

            clfs = {}
            clfs['Bagging'] = {'clf': BaggingClassifier(), 'name': 'Bagging'}
            clfs['Gradient Boosting'] = {'clf': GradientBoostingClassifier(), 'name': 'Gradient Boosting'}
            clfs['XGBoost'] = {'clf': XGBClassifier(), 'name': 'XGBoost'}
            clfs['Random Forest'] = {'clf': RandomForestClassifier(n_estimators=100, n_jobs=-1),
                                     'name': 'Random Forest'}
            clfs['Decision Tree'] = {'clf': DecisionTreeClassifier(), 'name': 'Decision Tree'}
            clfs['Extra Tree'] = {'clf': ExtraTreesClassifier(n_jobs=-1), 'name': 'Extra Tree'}

            clfs['KNN'] = {'clf': KNeighborsClassifier(n_jobs=-1), 'name': 'KNN'}
            # clfs['NN'] = {'clf': MLPClassifier(), 'name': 'MLPClassifier'
            # clfs['LR'] = {'clf': LogisticClassifier(), 'name': 'LR'}
            clfs['SVM'] = {'clf': SVC(gamma='auto'), 'name': 'SVM'}

            for item in clfs:
                
                if scoring == 'AUC' and np.unique(Target).shape[0] > 2:
                    scoring = self.AUC
                    score = 'AUC'
                else :
                    score = 'AUC'
                    scoring = 'roc_auc'
                
                Score = cross_val_score(clfs[item]['clf'], np.asarray(X_tr[0:n]), np.ravel(Y_tr[0:n]),
                                        cv=self.cv, scoring=scoring)
               
                Score_mean = Score.mean()
                STD2 = Score.std() * 2

                clfs[item]['score'] = Score  # roc_auc
                clfs[item]['mean'] = Score_mean
                clfs[item]['std2'] = STD2

                print("\n {}".format(item + ": %0.4f (+/- %0.4f)" % (clfs[item]['score'].mean(),
                                                                     clfs[item]['score'].std() * 2)))

            Best_clf = clfs[max(clfs.keys(), key=(lambda k: clfs[k]['mean']))]['name']


        elif self.type_esti == 'regressor':

            clfs = {}
            clfs['Bagging'] = {'clf': BaggingRegressor(), 'name': 'Bagging'}
            clfs['Gradient Boosting'] = {'clf': GradientBoostingRegressor(), 'name': 'Gradient Boosting'}
            clfs['XGBoost'] = {'clf': XGBRegressor(), 'name': 'XGBoost'}
            clfs['Random Forest'] = {'clf': RandomForestRegressor(n_estimators=100, n_jobs=-1),
                                     'name': 'Random Forest'}
            clfs['Decision Tree'] = {'clf': DecisionTreeRegressor(), 'name': 'Decision Tree'}
            clfs['Extra Tree'] = {'clf': ExtraTreesRegressor(n_jobs=-1), 'name': 'Extra Tree'}
            clfs['KNN'] = {'clf': KNeighborsRegressor(n_jobs=-1), 'name': 'KNN'}
            # clfs['NN'] = {'clf': MLPClassifier(), 'name': 'MLPClassifier'
            # clfs['LR'] = {'clf': LogisticClassifier(), 'name': 'LR'}
            clfs['SVM'] = {'clf': SVR(gamma='auto'), 'name': 'SVM'}

            for item in clfs:
                # print(Y_tr[0:30])
                Score = cross_val_score(clfs[item]['clf'], np.asarray(X_tr[0:n]), np.array(np.ravel(Y_tr[0:n])),
                                        ########""
                                        cv=self.cv, scoring=scoring)
                Score_mean = Score.mean()
                STD2 = Score.std() * 2

                clfs[item]['score'] = Score  # roc_auc
                clfs[item]['mean'] = Score_mean
                clfs[item]['std2'] = STD2

                print("\n {}".format(item + ": %0.4f (+/- %0.4f)" % (clfs[item]['score'].mean(),
                                                                     clfs[item]['score'].std() * 2)))

            Best_clf = clfs[max(clfs.keys(), key=(lambda k: clfs[k]['mean']))]['name']

        if self.grid:
            # print('grid = True')

            if params == False:
                # print('params = False')

                # print(Best_clf)

                if self.hard_grid == False:

                    if Best_clf == 'Extra Tree':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [100, 300, 600],
                                      'criterion': ['mse', 'mae'],
                                      'max_depth': [None, 5, 10]}

                        else:

                            params = {'n_estimators': [100, 300, 600],
                                      'criterion': ['gini', 'entropy'],
                                      'max_depth': [None, 5, 10]}

                    if Best_clf == 'Gradient Boosting':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [100, 300, 600],
                                      'max_depth': [5, 10, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['ls', 'lad']}
                        else:

                            params = {'n_estimators': [100, 300, 600],
                                      'max_depth': [5, 10, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['deviance', 'exponential']}


                    elif Best_clf == 'Random Forest':
                        #  print('Best_clf = dt ou rf')

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [10, 100, 300],
                                      'max_depth': [5, 10, None],
                                      'criterion': ['mse', 'mae']}

                        else:

                            params = {'n_estimators': [10, 100, 300],
                                      'max_depth': [5, 10, None],
                                      'criterion': ['gini', 'entropy']}

                    elif Best_clf == 'Decision Tree':

                        if self.type_esti == 'regressor':

                            params = {'max_depth': [5, 10, 50, None],
                                      'criterion': ['mse', 'friedman_mse', 'mae']}

                        else:

                            params = {'max_depth': [5, 10, 50, None],
                                      'criterion': ['gini', 'entropy']}


                    elif Best_clf == 'XGBoost':
                        # print('Best_clf = xgb')

                        params = {'eta': [.01, .1, .3],
                                  'max_depth': [5, 10, 15],
                                  'gamma': [0, .1, .01]}

                    elif Best_clf == 'Bagging':
                        # print('best_clf = bag)')

                        params = {'n_estimators': [100, 300, 600]}

                    elif Best_clf == 'KNN':

                        params = {'n_neighbors': [2, 5, 10, 30, 40],
                                  'p': [1, 2]}

                    elif Best_clf == 'SVM':

                        params = {'C': {1, .5, .1, 5},
                                  'tol': [.01, .001, .1, .0001]}



                else:

                    if Best_clf == 'Extra Tree':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'criterion': ['mae', 'mse'],
                                      'max_depth': [None, 5, 10, 15, 20, 25]}

                        else:

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'criterion': ['gini', 'entropy'],
                                      'max_depth': [None, 5, 10, 15, 20, 25]}

                    if Best_clf == 'Gradient Boosting':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 25, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['ls', 'lad', 'huber', 'quantile'],
                                      'criterion': ['mse', 'friedman_mse']}
                        else:

                            params = {'n_estimators': [100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 25, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['deviance', 'exponential'],
                                      'criterion': ['mse', 'friedman_mse']}


                    elif Best_clf == 'Random Forest':
                        #  print('Best_clf = dt ou rf')

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 20, 25, None],
                                      'criterion': ['mse', 'mae']}

                        else:

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 20, 25],
                                      'criterion': ['gini', 'entropy']}

                    elif Best_clf == 'Decision Tree':

                        if params == 'regressor':

                            params = {'max_depth': [5, 10, 50, 100, None],
                                      'criterion': ['mse', 'friedman_mse', 'mae'],
                                      'splitter': ['best', 'random']}

                        else:

                            params = {'max_depth': [5, 10, 50, 100, None],
                                      'criterion': ['gini', 'entropy'],
                                      'splitter': ['best', 'random']}


                    elif Best_clf == 'XGBoost':
                        # print('Best_clf = xgb')

                        params = {'eta': [0.001, .01, .1, .3, 1],
                                  'max_depth': [5, 10, 15, 20, 25],
                                  'gamma': [0, .1, .01, .001]}

                    elif Best_clf == 'Bagging':
                        # print('best_clf = bag)')

                        params = {'n_estimators': [100, 300, 600, 1000, 1200, 1500]}

                    elif Best_clf == 'KNN':

                        params = {'n_neighbors': [2, 5, 10, 30, 40, 70, 100],
                                  'p': [1, 2, 3]}

                    elif Best_clf == 'SVM':

                        params = {'C': {1, .5, .1, 5, .01, .001},
                                  'tol': [.01, .001, .1, .0001, 1],
                                  'kernel': ['rbf', 'linear', 'poly', 'sigmoid', 'precomputed']}

            if self.hard_grid:
                print('\n Searching for the best hyperparametres of {} using hard_grid on {} data among : \n'.format(
                    Best_clf, n_grid))

            else:
                print('\n Searching for the best hyperparametres of {} on {} data among : \n'.format(Best_clf, n_grid))
            print('{} \n'.format(params))
            # print('Starting GridSearchCV using {} Classifier with {} folds \n'.format(Best_clf, cv_grid))

            
        
            clf = clfs[max(clfs.keys(), key=(lambda k: clfs[k]['mean']))]['clf']

            if loss == 'AUC' and np.unique(Target).shape[0] > 2:
                gr = GridSearchCV(clf, param_grid=params, cv=self.cv_grid, scoring=scoring, 
                                verbose=1, refit=True, iid=True)
            else :
                gr = GridSearchCV(clf, param_grid=params, cv=self.cv_grid, scoring=scoring, 
                                    verbose=1, refit=True, iid=True, n_jobs = -1)

            gr.fit(X_tr[0:n_grid], np.ravel(Y_tr[0:n_grid]))

            # print(' Best score :', gr.best_score_,   '\n Using these parametres :', gr.best_params_)

            #####

            print('\n Finally, the best estimator is : {} {}'.format(Best_clf, self.type_esti))
            print('\n Using these hyperparametres : {}'.format(gr.best_params_))

            print('\n With this {} score : {}'.format(loss, gr.best_score_))

            # return (gr) !!!!!!!
        else:
            print('\n Best {} : {}'.format(self.type_esti, Best_clf))

    def grid(self, clf, params, cv=3, n=100000):

        X_tr, X_te, Y_tr, Y_te = train_test_split(self.Data, self.Target, random_state=0, test_size=1 / 3)

        gr = GridSearchCV(clf, param_grid=params, cv=cv, scoring=self.AUC, n_jobs=-1,
                          verbose=1, refit=True, iid=True);

        gr.fit(X_tr[0:n], np.ravel(Y_tr[0:n]))

        # print(' Best score :', gr.best_score_,   '\n Using this parametres :', gr.best_params_, '\n With :', clf)
        print(' Best score on Train:', gr.best_score_, '\n Using this parametres :', gr.best_params_,
              '\n With : \n {} '.format(clf))
        return gr

    
    
    def Transform(self, Data, value=0, ID = 'ID'):
        
        Test = Data.copy()
        
        if ID != None:

            Test.drop([ID], axis=1, inplace=True)

        if type(value) == int:
            Test.fillna(value, inplace=True)

        elif value == 'bfill':
            Test.fillna('bfill', inplace=True)

        elif value == 'ffill':
            Test.fillna('ffill', inplace=True)

        for i in Test.columns:  ###########
            if Test[i].dtype == float:
                Test[i] = Test[i].astype('int')

            elif Test[i].dtype == object:
                encoder = LabelEncoder()
                encoder.fit(list(Test[i]))
                Test[i] = encoder.transform(list(Test[i]))
        return(Test)
	

    def pred(self, Test, gr, prob=False, same=True, ID='ID', value=0):  #

        # Test.drop([ID], axis = 1, inplace = True)
        Pred = pd.DataFrame()

        if same == False:

            Test.drop([ID], axis=1, inplace=True)

            if type(value) == int:
                Test.fillna(value, inplace=True)

            elif value == 'bfill':
                Test.fillna('bfill', inplace=True)

            elif value == 'ffill':
                Test.fillna('ffill', inplace=True)

            for i in Test.columns:
                if Test[i].dtype == float:
                    Test[i] = Test[i].astype('int')

                elif Test[i].dtype == object:
                    encoder = LabelEncoder()
                    encoder.fit(list(Test[i]))
                    Test[i] = encoder.transform(list(Test[i]))

        if prob == False:
            # Pred[ID] = Test[ID]
            Pred['Target'] = gr.predict(Test)
            return (Pred)

        else:
            return (gr.predict_proba(Test))

    # else :
    #    return(gr.predict_proba(self.feature_eng(Data, value , ID)))








### Multiclass

In [2]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train.csv", sep = ',')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target.csv", sep = ';')

### Binary

In [19]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train1.csv", sep = ';')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target1.csv", sep = ';')

### Regression

In [ ]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\input_training.csv", sep = ';')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target_Engie.csv", sep = ';')

In [24]:
clf = BestEstimator(type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 1000,
            value_nan = 0,
            view_nan = True,
           params = False,
        scoring = 'AUC') 

Missing Values :

                            Total         %
1_diffClosing stocks(kmt)     261  2.569151
2_diffClosing stocks(kmt)     257  2.529777
3_diffClosing stocks(kmt)     253  2.490403
4_diffClosing stocks(kmt)     249  2.451029
5_diffClosing stocks(kmt)     245  2.411655
6_diffClosing stocks(kmt)     241  2.372281
7_diffClosing stocks(kmt)     238  2.342750
8_diffClosing stocks(kmt)     235  2.313220
9_diffClosing stocks(kmt)     232  2.283689
10_diffClosing stocks(kmt)    228  2.244315
11_diffClosing stocks(kmt)    224  2.204941
12_diffClosing stocks(kmt)    220  2.165567
1_diffImports(kmt)            151  1.486367
2_diffImports(kmt)            148  1.456836
3_diffImports(kmt)            145  1.427306
4_diffImports(kmt)            142  1.397775
5_diffImports(kmt)            139  1.368245
6_diffImports(kmt)            136  1.338714
7_diffImports(kmt)            133  1.309184
8_diffImports(kmt)            130  1.279654
9_diffImports(kmt)            127  1.250123
10_diffImports

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   22.5s finished



 Finally, the best estimator is : Random Forest classifier

 Using these hyperparametres : {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 100}

 With this AUC score : 0.7436210162248299


In [40]:
clf = BestEstimator(type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

clf.Transform(Train)

,month,country,1_diffClosing stocks(kmt),1_diffExports(kmt),1_diffImports(kmt),1_diffRefinery intake(kmt),1_diffWTI,1_diffSumClosing stocks(kmt),1_diffSumExports(kmt),1_diffSumImports(kmt),...,12_diffClosing stocks(kmt),12_diffExports(kmt),12_diffImports(kmt),12_diffRefinery intake(kmt),12_diffWTI,12_diffSumClosing stocks(kmt),12_diffSumExports(kmt),12_diffSumImports(kmt),12_diffSumProduction(kmt),12_diffSumRefinery intake(kmt)
0,5,52,0,237,0,33,-5,10426,12135,20527,...,0,0,0,14,2,673,-13007,-5932,-17130,-13151
1,12,69,28,-34,60,7,-5,195,7500,6335,...,-43,-129,-67,-16,-5,3217,-6563,-3587,-13053,-13005
2,5,74,0,0,-29,-85,-5,10426,12135,20527,...,0,0,-17,-13,2,673,-13007,-5932,-17130,-13151
3,11,34,175,91,0,339,7,-2247,-6806,-10986,...,81,-579,0,16,-8,-2929,1072,504,-4827,3962
4,7,2,-550,251,0,-49,-12,-2652,2165,-4491,...,-1127,-512,0,-2,7,2513,-1732,-4827,-7242,-13079
5,12,18,-5,0,54,0,-6,4667,5818,9926,...,-93,0,-159,4,-7,-321,-2198,-3970,-8308,-9271
6,2,74,0,0,-330,14,7,-2222,8847,10787,...,0,0,-189,-40,-12,371,-9245,-11594,-14411,-10187
7,10,65,-11,0,-49,-13,-1,-795,-6970,15436,...,8,0,213,221,4,-6293,11736,9298,10007,10394
8,5,39,0,-769,-479,0,5,-24272,10527,16947,...,0,0,0,0,6,-674,-4264,-11110,-26769,-23634
9,11,3,206,-159,0,-373,-7,-321,-2198,-3970,...,155,104,0,-70,-4,-4976,2226,1150,-2947,1641


In [8]:
Reg = BestEstimator(type_esti = 'regressor', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

Reg.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 1000,
            value_nan = 0,
            view_nan = True,
           params = False,
       scoring = 'explained_variance') 

Missing Values :

                                Total          %
Grid_voltage                   101322  16.411451
Grid_voltage_std               101322  16.411451
Grid_voltage_max               101322  16.411451
Grid_voltage_min               101322  16.411451
Gearbox_inlet_temperature        8064   1.306152
Generator_converter_speed        8064   1.306152
Generator_converter_speed_min    8064   1.306152
Generator_converter_speed_max    8064   1.306152
Generator_converter_speed_std    8064   1.306152
Gearbox_inlet_temperature_min    8064   1.306152
Gearbox_inlet_temperature_max    8064   1.306152
Gearbox_inlet_temperature_std    8064   1.306152
Absolute_wind_direction_c          72   0.011662
Nacelle_angle_c                    72   0.011662 

NaN data filled by 0 

Searching for the best regressor on 1000 data using explained_variance loss... 


 Bagging: 0.9677 (+/- 0.0258)

 Gradient Boosting: 0.9766 (+/- 0.0081)

 XGBoost: 0.9761 (+/- 0.0139)

 Random Forest: 0.9703 (+/- 0.0206)



[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  5.2min finished



 Finally, the best estimator is : Gradient Boosting regressor

 Using these hyperparametres : {'learning_rate': 0.1, 'loss': 'lad', 'max_depth': 10, 'n_estimators': 600}

 With this explained_variance score : 0.9707338413345542
